## Data preprocessing

This part is used to perform data preprocessing including:

-  Filter stations within 10km of Zurich HB
- Find timetable for each unique identified trip id
- Find delay distribution of trips, which will be used in later Monte-Carlo sampling

### 1 Find Stations within 10km of Zurich HB
In this part we extract train information where the train stops are within 10km distance to Zurich stop.

In [5]:
import pandas as pd
import numpy as np
import os

In [2]:
DATE = '201804'
DATA_PATH = './data/2018-04/'

In [69]:
geo = pd.read_csv('./data/BFKOORD_GEO', sep="%", header=None,error_bad_lines=False)
geo.columns = ['data','station_name']
geo.name = geo.station_name.apply(str.lstrip).apply(str.rstrip)

/anaconda3/envs/semester/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


In [70]:
geo[['station_number','longtitude','latitude','height']] = geo.data.str.split(expand=True)#.apply(float)
geo.drop('data',axis=1,inplace=True)

In [5]:
# station in Zurich
zurich = geo[geo.station_number=="8503000"].reset_index(drop=True)
zurich

,name,station_number,longtitude,latitude,height
0,Zürich HB,8503000,8.540192,47.378177,408


We will focus on all the stops within 10km of the Zurich train station.

We calculated distance using this formula:

$$ DISTANCE = 2* arcsin\sqrt{sin^2\frac{a}{2}+cos(Lat1)*cos(Lat2)*sin^2\frac{b}{2}} * Earth Radius $$
$$ a = Lat1-Lat2, b = Lon1 - Lon2 $$

In [6]:
# Define a function to compute the distance with the longtitude and the latitude
from math import sin, cos, sqrt, atan2, radians,asin
def compute_distance(point_1_lat, point_1_lon, point_2_lat=zurich.latitude[0], point_2_lon=zurich.longtitude[0]):
    # approximate radius of earth in km
    R = 6378.137 # earth radius

    lat1 = radians(float(point_1_lat))
    lon1 = radians(float(point_1_lon))
    lat2 = radians(float(point_2_lat))
    lon2 = radians(float(point_2_lon))

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))

    distance = R * c
    return np.round(distance,3) # return distance in kilometres

In [7]:
distance = []
for log,lat in zip(geo.longtitude,geo.latitude):
    distance.append(compute_distance(lat,log))

In [8]:
geo['distance'] = distance

In [11]:
zurich_neigh_geo = geo.loc[geo.distance<=10].reset_index(drop=True)
zurich_neigh_geo.head()

,name,station_number,longtitude,latitude,height,distance
0,Zimmerberg-Basistunnel,0000176,8.521961,47.351679,0,3.254
1,Urdorf,8502220,8.434713,47.390882,442,8.075
2,Birmensdorf ZH,8502221,8.437543,47.357432,488,8.076
3,Bonstetten-Wettswil,8502222,8.468175,47.325896,528,7.961
4,Urdorf Weihermatt,8502229,8.430330,47.380971,456,8.287


In [12]:
# load all files in that folder
info = [os.path.join(DATA_PATH,file) for file in os.listdir(DATA_PATH) if file.endswith('.csv') ]

In [13]:
data = pd.concat((pd.read_csv(f,sep=';') for f in info))

/anaconda3/envs/semester/lib/python3.6/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (6,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.


In [14]:
# filter out the station within 10km of Zurich HB
zurich_neigh_train = data.loc[data.HALTESTELLEN_NAME.isin(zurich_neigh_geo.name.tolist())].reset_index(drop=True)

In [17]:
data_col = ['BETRIEBSTAG','FAHRT_BEZEICHNER','BETREIBER_ABK', 'BETREIBER_NAME','PRODUKT_ID',
            'LINIEN_ID','LINIEN_TEXT','VERKEHRSMITTEL_TEXT','ZUSATZFAHRT_TF','FAELLT_AUS_TF',
            'HALTESTELLEN_NAME','ANKUNFTSZEIT','AN_PROGNOSE','AN_PROGNOSE_STATUS','ABFAHRTSZEIT',
            'AB_PROGNOSE','AB_PROGNOSE_STATUS','DURCHFAHRT_TF']

In [18]:
zurich_neigh_train = zurich_neigh_train[data_col]

In [19]:
# rename the German name into English
zurich_neigh_train.rename(columns={'BETRIEBSTAG':'date_of_trip',
                               'FAHRT_BEZEICHNER':'identifies_of_trip',
                               'BETREIBER_ID':'GO_number',
                               'BETREIBER_ABK':'operator_first_name',
                               'PRODUKT_ID':'product_id',
                               'BPUIC':'station_number',
                               'UMLAUF_ID':'circulation ID',
                               'BETREIBER_NAME':'operator_name',
                               'PRODUCT_ID':'type_of_transport,',
                               'LINIEN_ID':'train_number',
                               'LINIEN_TEXT':'service_type',
                               'VERKEHRSMITTEL_TEXT':'service_type_2',
                               'ZUSATZFAHRT_TF':'additional_trip',
                               'FAELLT_AUS_TF':'failed_trip',
                               'HALTESTELLEN_NAME':'station_name',
                               'ANKUNFTSZEIT':'arrival_time',
                               'AN_PROGNOSE':'actual_arrival_time',
                               'AN_PROGNOSE_STATUS':'arrival_status',
                               'ABFAHRTSZEIT':'departure_time',
                               'AB_PROGNOSE':"actual_departure_time",
                               'AB_PROGNOSE_STATUS':'departure_status',
                               'DURCHFAHRT_TF':'not_stop'}, 
                      inplace=True)

In [20]:
zurich_neigh_train.head()

,date_of_trip,identifies_of_trip,operator_first_name,operator_name,product_id,train_number,service_type,service_type_2,additional_trip,failed_trip,station_name,arrival_time,actual_arrival_time,arrival_status,departure_time,actual_departure_time,departure_status,not_stop
0,09.04.2018,85:11:10:002,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,EC,False,False,Zürich HB,09.04.2018 21:51,09.04.2018 21:51:18,GESCHAETZT,NaN,NaN,PROGNOSE,False
1,09.04.2018,85:11:11:001,SBB,Schweizerische Bundesbahnen SBB,Zug,11,EC,EC,False,False,Zürich HB,NaN,NaN,PROGNOSE,09.04.2018 06:09,09.04.2018 06:10:58,GESCHAETZT,False
2,09.04.2018,85:11:12:001,SBB,Schweizerische Bundesbahnen SBB,Zug,12,EC,EC,False,False,Zürich HB,09.04.2018 10:51,09.04.2018 10:51:06,GESCHAETZT,NaN,NaN,PROGNOSE,False
3,09.04.2018,85:11:1251:001,SBB,Schweizerische Bundesbahnen SBB,Zug,1251,IC3,IC,False,False,Zürich HB,09.04.2018 07:00,09.04.2018 06:59:40,GESCHAETZT,NaN,NaN,PROGNOSE,False
4,09.04.2018,85:11:1252:001,SBB,Schweizerische Bundesbahnen SBB,Zug,1252,IC,IC,False,False,Zürich HB,09.04.2018 21:23,09.04.2018 21:22:44,GESCHAETZT,09.04.2018 21:36,09.04.2018 21:37:29,GESCHAETZT,False


In [23]:
# save the .csv file
zurich_neigh_train.to_csv('neighbour_%s.csv'%DATE,index=False)

----
### 2 Find timetable for each unique identified trip id
In original dataset, each row only contains one station but for each trip there are many stations. So in this part we group the dataset with their date and trip id, and aggregate the station name, scheduel arrival time, actual arrival time, scheduel departure time and  actual departure time.

In [24]:
from datetime import datetime
def Time(x):
    """
    Return only the time when the trip takes place.
    """
    if type(x)==float:
        return 'NaN'
    else:
        return x.split()[1]
    
def weekday(x):
    """
    Return the day of week of the trip.
    """
    return datetime.strptime(x, '%d.%m.%Y').weekday()

In [25]:
zurich_neigh_train.arrival_time = zurich_neigh_train.arrival_time.apply(Time)
zurich_neigh_train.actual_arrival_time = zurich_neigh_train.actual_arrival_time.apply(Time)
zurich_neigh_train.departure_time = zurich_neigh_train.departure_time.apply(Time)
zurich_neigh_train.actual_departure_time = zurich_neigh_train.actual_departure_time.apply(Time)
zurich_neigh_train.service_type = zurich_neigh_train.service_type.apply(str)

In [26]:
zurich_neigh_train['Timetable'] = zurich_neigh_train['station_name']+' '+zurich_neigh_train['arrival_time']+' '\
        +zurich_neigh_train['actual_arrival_time']\
        +' '+zurich_neigh_train['departure_time']+' '+zurich_neigh_train['actual_departure_time']
zurich_neigh_train.head()

,date_of_trip,identifies_of_trip,operator_first_name,operator_name,product_id,train_number,service_type,service_type_2,additional_trip,failed_trip,station_name,arrival_time,actual_arrival_time,arrival_status,departure_time,actual_departure_time,departure_status,not_stop,Timetable
0,09.04.2018,85:11:10:002,SBB,Schweizerische Bundesbahnen SBB,Zug,10,EC,EC,False,False,Zürich HB,21:51,21:51:18,GESCHAETZT,NaN,NaN,PROGNOSE,False,Zürich HB 21:51 21:51:18 NaN NaN
1,09.04.2018,85:11:11:001,SBB,Schweizerische Bundesbahnen SBB,Zug,11,EC,EC,False,False,Zürich HB,NaN,NaN,PROGNOSE,06:09,06:10:58,GESCHAETZT,False,Zürich HB NaN NaN 06:09 06:10:58
2,09.04.2018,85:11:12:001,SBB,Schweizerische Bundesbahnen SBB,Zug,12,EC,EC,False,False,Zürich HB,10:51,10:51:06,GESCHAETZT,NaN,NaN,PROGNOSE,False,Zürich HB 10:51 10:51:06 NaN NaN
3,09.04.2018,85:11:1251:001,SBB,Schweizerische Bundesbahnen SBB,Zug,1251,IC3,IC,False,False,Zürich HB,07:00,06:59:40,GESCHAETZT,NaN,NaN,PROGNOSE,False,Zürich HB 07:00 06:59:40 NaN NaN
4,09.04.2018,85:11:1252:001,SBB,Schweizerische Bundesbahnen SBB,Zug,1252,IC,IC,False,False,Zürich HB,21:23,21:22:44,GESCHAETZT,21:36,21:37:29,GESCHAETZT,False,Zürich HB 21:23 21:22:44 21:36 21:37:29


In [27]:
# for each identify of trip, it has its own product_id, train_number, service type
# we drop duplicate to get the unique information
info = zurich_neigh_train[['date_of_trip','identifies_of_trip','product_id',
                           'train_number','service_type',
                           'additional_trip','not_stop']].drop_duplicates()
info.dropna(subset=['product_id'],inplace=True)
info.drop_duplicates(subset=['date_of_trip','identifies_of_trip'],inplace=True)
info.head()

,date_of_trip,identifies_of_trip,product_id,train_number,service_type,additional_trip,not_stop
0,09.04.2018,85:11:10:002,Zug,10,EC,False,False
1,09.04.2018,85:11:11:001,Zug,11,EC,False,False
2,09.04.2018,85:11:12:001,Zug,12,EC,False,False
3,09.04.2018,85:11:1251:001,Zug,1251,IC3,False,False
4,09.04.2018,85:11:1252:001,Zug,1252,IC,False,False


In [29]:
df = zurich_neigh_train[['date_of_trip','identifies_of_trip','Timetable']]\
                        .groupby(['date_of_trip','identifies_of_trip']).aggregate(lambda x: tuple(x))
df = df.loc[df.Timetable.apply(len)>1].reset_index()
df.reset_index(inplace=True,drop=True)

In [30]:
df = df.merge(info) # combine product_id, train_number and service_type

In [31]:
df['dayOfWeek'] = df.date_of_trip.apply(weekday) # return the day of week of trip date

In [87]:
def convertsecond(x):
    """
    In this function we convert the string into seconds.
    """
    x = x.split(':')
    if len(x)==1:
        return np.nan
    if len(x)==2:
        return int(x[0])*3600+int(x[1])*60
    if len(x)==3:
        return int(x[0])*3600+int(x[1])*60+int(x[2])

def formatfunc(x):
    """
    This function is used to return standard format of timetabel: 
    [station name, schedule arrival time, actual arrival time, schdule departure time, actual departure time]
    """
    tmp_list =[]
    for ele in x:
        tmp = ele.split(' ')
        station = ' '.join(tmp[:-4])
        tmp_list.append([station,
                         convertsecond(tmp[-4]),
                         convertsecond(tmp[-3]),
                         convertsecond(tmp[-2]),
                         convertsecond(tmp[-1])])    
    return tmp_list

In [33]:
df['Timetable'] = df.Timetable.apply(formatfunc)

In [34]:
df.head()

,date_of_trip,identifies_of_trip,Timetable,product_id,train_number,service_type,additional_trip,not_stop,dayOfWeek
0,01.04.2018,85:11:13710:001,"[[Zürich HB, nan, nan, 3600, 3656], [Zürich Ha...",Zug,13710,SN1,False,False,6
1,01.04.2018,85:11:13711:001,"[[Zürich HB, nan, nan, 3900, 3933], [Zürich St...",Zug,13711,SN1,False,False,6
2,01.04.2018,85:11:13712:001,"[[Dietlikon, 6360, 6391, 6360, 6443], [Stettba...",Zug,13712,SN1,False,False,6
3,01.04.2018,85:11:13713:001,"[[Glanzenberg, 6720, 7070, 6720, 7120], [Schli...",Zug,13713,SN1,False,False,6
4,01.04.2018,85:11:13714:001,"[[Dietlikon, 9960, 10412, 9960, 10471], [Stett...",Zug,13714,SN1,False,False,6


In [36]:
df.to_csv('grouped_%s.csv'%DATE,index=False)

---
### 3 Delay distribution of trips
For each station, each trip has its only delay distribution (actual time - schedule time, counted in seconds). In this part we extract distrbutions of each trip of each station.


In [8]:
df = pd.read_csv('grouped_201804.csv')

In [59]:
nan = np.nan

In [60]:
df.Timetable = df.Timetable.map(lambda x: eval(x))

In [61]:
import math
def timefunc(x):
    """
    This function is used to return arrival and depature differnence.
    """
    x = x[1:]
    if math.isnan(x[0]) | math.isnan(x[1]):
        arr = 0
    else:
        arr = x[1]-x[0]
    if math.isnan(x[2]) | math.isnan(x[3]):
        dep = 0
    else:
        dep = x[3]-x[2]
    return arr,dep

In [62]:
dict_ = {}
for i in range(len(df)):
    if df.additional_trip.loc[i] == True: # filter out additional trips
        continue
    trip_id = df.identifies_of_trip.loc[i]
    stations = df.Timetable.loc[i]
    stations = [x[0] for x in stations]
    delay_time = [timefunc(x) for x in df.Timetable.loc[i]]
    for x,y in zip(stations,delay_time):
        if x in dict_.keys():
            if trip_id in dict_[x].keys():
                dict_[x][trip_id]['arr'].append(y[0])
                dict_[x][trip_id]['dep'].append(y[1])
            else:
                dict_[x][trip_id] = {'arr':[y[0]],'dep':[y[1]]}
        else:
            dict_[x] = {trip_id:{'arr':[y[0]],'dep':[y[1]]}}

In [68]:
trip_list = []
for station in dict_.keys():
    for trip in dict_[station].keys():
        arr = dict_[station][trip]['arr']
        dep = dict_[station][trip]['dep']
        trip_list.append([station,trip,arr,dep])

In [69]:
df_dense = pd.DataFrame(trip_list)

In [70]:
df_dense.columns = ['station_name','identifies_of_trip','arr_delay','dep_delay']

In [71]:
df_dense.head(10)

,station_name,identifies_of_trip,arr_delay,dep_delay
0,Zürich HB,85:11:13710:001,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[56, 40, 99, 66, 44, 534, 73, 39, 81, 58]"
1,Zürich HB,85:11:13711:001,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[33, 30, 75, 42, 0, 0, 31, 65, 87, 54]"
2,Zürich HB,85:11:13712:001,"[-36, -45, 90, 73, 28, 209, 2, 263, 211, 9]","[56, 33, 104, 74, 62, 206, 55, 264, 202, 62]"
3,Zürich HB,85:11:13713:001,"[427, 60, 255, 131, 71, 140, 100, 151, 142, 107]","[403, 31, 224, 134, 45, 107, 67, 138, 113, 90]"
4,Zürich HB,85:11:13714:001,"[430, -40, 186, 31, -44, -33, -22, 39, -29, -52]","[430, 53, 173, 117, 133, 65, 43, 55, 40, 74]"
5,Zürich HB,85:11:13715:001,"[703, 40, 138, 153, 182, 606, 181, 163, 186, 64]","[670, 44, 131, 122, 141, 566, 138, 134, 146, 107]"
6,Zürich HB,85:11:13716:001,"[338, -56, 44, -7, -11, 245, -11, -36, 35, 17]","[300, 64, 49, 57, 53, 209, 30, 51, 40, 74]"
7,Zürich HB,85:11:13717:001,"[162, 4, 351, 117, 89, 85, 87, 413, 293, 38]","[160, 21, 319, 82, 59, 127, 41, 390, 247, 33]"
8,Zürich HB,85:11:13750:001,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[54, 23, 35, 29, 123, 47, 43, 43, 35, 35]"
9,Zürich HB,85:11:13751:001,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[55, 20, 53, 55, 57, 24, 41, 50, 29, 55]"


In [72]:
df_dense.to_csv('grouped_dense_201804.csv',index=False) # save distribution of one month

In [76]:
# then we combine all monthes' distribution
DATA_PATH = './'
info = [os.path.join(DATA_PATH,file) for file in os.listdir(DATA_PATH) if (file.endswith('.csv'))&(file.startswith('grouped_dense_')) ]

In [77]:
info

['./grouped_dense_201712.csv',
 './grouped_dense_201710.csv',
 './grouped_dense_201711.csv',
 './grouped_dense_201709.csv',
 './grouped_dense_201804.csv',
 './grouped_dense_201802.csv',
 './grouped_dense_201803.csv',
 './grouped_dense_201801.csv']

In [78]:
data = pd.concat((pd.read_csv(f) for f in info))

In [79]:
df = data.groupby(['station_name','identifies_of_trip']).aggregate(lambda x: tuple(x))

In [80]:
df.reset_index(inplace=True)

In [81]:
def concatTuple(x):
    y = []
    for y_ in x:
        x_ = y_.replace('[','').replace(']','')
        y.append(x_)
    y = ', '.join(y)
    return '['+y+']'

In [82]:
df.arr_delay = df.arr_delay.apply(concatTuple)

In [83]:
df.dep_delay = df.dep_delay.apply(concatTuple)

In [84]:
df.head()

,station_name,identifies_of_trip,arr_delay,dep_delay
0,"Adlikon b. R., Leematten",85:773:17869-11451-1,"[40, 0, 0, -69, 131, 118, 0, 18, 6, 75, 35, 29...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"Adlikon b. R., Leematten",85:773:17870-11451-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[40, 0, 0, 0, 143, 118, 0, 18, 6, 65, 35, 29, ..."
2,"Adlikon b. R., Leematten",85:773:17873-16451-1,"[0, -59, 104, 7]","[0, 0, 0, 0]"
3,"Adlikon b. R., Leematten",85:773:17874-11451-1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 52, 0, 6, 0, 1, 0, 0, 113, 68, 10, 0, 0..."
4,"Adlikon b. R., Leematten",85:773:17877-16451-1,"[-16, -62, 35, -49]","[0, 0, 0, 0]"


In [45]:
#df.to_csv('group_dense.csv')

---
### 4 Schedule of trips

Then we want to extract the schedule of trips. And to avoid situation where past tripa are cancelled, we use the newest data (201804) and discuss under three situation:

- Weekday schedule (take 2018/04/30, Monday as representative)
- Weekend schedule (take 2018/04/28, Saturday as representative)
- Holiday schedule (take 2018/04/29, Sunday as representative)

In [81]:
df_0430 = df.loc[df.date_of_trip=='30.04.2018'].reset_index(drop=True).copy()
df_0430

,date_of_trip,identifies_of_trip,Timetable,product_id,train_number,service_type,additional_trip,not_stop,dayOfWeek
0,30.04.2018,85:11:1507:002,"[[Zürich HB, 23400, 23593, 23940, 23984], [Zür...",Zug,1507,IC5,False,False,0
1,30.04.2018,85:11:1509:003,"[[Zürich HB, 27000, 27113, 27540, 27582], [Zür...",Zug,1509,IC5,False,False,0
2,30.04.2018,85:11:1510:003,"[[Zürich Flughafen, 25860, 25906, 25980, 26102...",Zug,1510,IC5,False,False,0
3,30.04.2018,85:11:1511:003,"[[Zürich HB, 30600, 30690, 31140, 31223], [Zür...",Zug,1511,IC5,False,False,0
4,30.04.2018,85:11:1512:003,"[[Zürich Flughafen, 29460, 29451, 29580, 29710...",Zug,1512,IC5,False,False,0
5,30.04.2018,85:11:1513:003,"[[Zürich HB, 34200, 34329, 34740, 34776], [Zür...",Zug,1513,IC5,False,False,0
6,30.04.2018,85:11:1514:003,"[[Zürich Flughafen, 33060, 33121, 33180, 33300...",Zug,1514,IC5,False,False,0
7,30.04.2018,85:11:1515:003,"[[Zürich HB, 37800, 37948, 38340, 38411], [Zür...",Zug,1515,IC5,False,False,0
8,30.04.2018,85:11:1516:001,"[[Zürich Flughafen, 36660, 36612, 36780, 36838...",Zug,1516,IC5,False,False,0
9,30.04.2018,85:11:1517:001,"[[Zürich HB, 41400, 41402, 41940, 41988], [Zür...",Zug,1517,IC5,False,False,0


In [82]:
dict_sched = {}
for i in range(len(df_0430)):
    if df_0430.additional_trip.loc[i] == True:
        continue
    trip_id = df_0430.identifies_of_trip.loc[i]
    stations = df_0430.Timetable.loc[i]
    stations = [x[0] for x in stations]
    delay_time = [(x[1],x[3]) for x in df_0430.Timetable.loc[i]]
    for x,y in zip(stations,delay_time):
        if x in dict_sched.keys():
            if trip_id in dict_sched[x].keys():
                dict_sched[x][trip_id]['arr'].append(y[0])
                dict_sched[x][trip_id]['dep'].append(y[1])
            else:
                dict_sched[x][trip_id] = {'arr':[y[0]],'dep':[y[1]]}
        else:
            dict_sched[x] = {trip_id:{'arr':[y[0]],'dep':[y[1]]}}

In [83]:
trip_list = []
for station in dict_sched.keys():
    for trip in dict_sched[station].keys():
        arr = dict_sched[station][trip]['arr'][0]
        dep = dict_sched[station][trip]['dep'][0]
        trip_list.append([station,trip,arr,dep])

In [84]:
df_sched = pd.DataFrame(trip_list)

In [85]:
df_sched.columns = ['station_name','identifies_of_trip','arrival_time','depature_time']

In [86]:
df_sched.head()

,station_name,identifies_of_trip,arrival_time,depature_time
0,Zürich HB,85:11:1507:002,23400.0,23940.0
1,Zürich HB,85:11:1509:003,27000.0,27540.0
2,Zürich HB,85:11:1510:003,26580.0,27000.0
3,Zürich HB,85:11:1511:003,30600.0,31140.0
4,Zürich HB,85:11:1512:003,30180.0,30600.0


In [87]:
df_sched.to_csv('schedule_holiday.csv',index=False)